<a href="https://colab.research.google.com/github/hbechara/HertieML/blob/main/Lesson9Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Today's Quest:
- Split the MNIST Data into a training set, validation set and test set. (I've done this part for you)
- Train various classifiers (Random Forest, Extra-Trees, SVM, etc).
- Try to combine them into an ensemble that outperforms each individual classifier.

# Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)



# Load Dataset and Split 

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

# Train various classifiers:

- I have made some suggestions. Pick your poison. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# training the classifier

svm_clr = LinearSVC(random_state=42)
svm_clr.fit(X_train, y_train)

log_clr = LogisticRegression(max_iter=200, random_state=42)
log_clr.fit(X_train, y_train)

nn_clr = MLPClassifier(random_state=42)
nn_clr.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=42, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
y_pred_svm = svm_clf.predict(X_val)
y_pred_log = log_clr.predict(X_val)
y_pred_nn = nn_clr.predict(X_val)

NameError: ignored

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_val, y_pred_log))


In [ ]:
print(accuracy_score(y_val, y_pred_svm))


In [ ]:
print(accuracy_score(y_val, y_pred_nn))

# Which of your estimators performed best?

- Compare on the validation set

# Combine your classifiers into an ensemble that outperforms them all on the validation set.

In [ ]:
from sklearn.ensemble import VotingClassifier

# Try some different combinations of the classifiers. Which combination works best?

Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image's class. Train a classifier on this new training set.

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    estimator.fit(X_train, y_train)

X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

NameError: ignored

# Thilos solution for Stacking exercise

In [ ]:
array_list = [y_pred_rnd, y_pred_ext, y_pred_mlp]
array_length = len(y_pred_rnd)
train_new = np.empty([0,array_length])

for y in array_list:
  train_new = np.vstack((train_new, y))

train_new


array([[5., 8., 2., ..., 7., 6., 7.],
       [5., 8., 2., ..., 7., 6., 7.],
       [5., 8., 2., ..., 7., 6., 7.]])

In [ ]:
train_corr = train_new.T

train_corr

array([[5., 5., 5.],
       [8., 8., 8.],
       [2., 2., 2.],
       ...,
       [7., 7., 7.],
       [6., 6., 6.],
       [7., 7., 7.]])

In [ ]:
rf_new = RandomForestClassifier(random_state=42)

rf_new.fit(train_corr, y_val)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

## On the test set

In [ ]:
y_test_rnd = rnd_clf.predict(X_test)
y_test_ext = ext_clf.predict(X_test)
y_test_mlp = mlp_clf.predict(X_test)

In [ ]:
y_test_rnd

array([8, 4, 8, ..., 3, 8, 3], dtype=uint8)

In [ ]:
array_list = [y_test_rnd, y_test_ext, y_test_mlp]
array_length = len(y_test_rnd)
test_new = np.empty([0,array_length])

for y in array_list:
  test_new = np.vstack((test_new, y))

test_new_transposed = test_new.T


In [ ]:
test_new_transposed

array([[8., 8., 8.],
       [4., 4., 4.],
       [8., 8., 8.],
       ...,
       [3., 3., 3.],
       [8., 8., 8.],
       [3., 3., 3.]])

In [ ]:
test_predict = rf_new.predict(test_new_transposed)
print(rf_new.__class__.__name__, accuracy_score(y_test, test_predict))

RandomForestClassifier 0.9674
